In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import numpy as np
from tensorflow.keras.optimizers import Adam
import re

# Load dataset

data = pd.read_csv("dataset_Chatbot.csv")

# Preprocess data
X = data['pattern'].astype(str)
y_emotion= data['tag'].astype(str)

#X = data['pattern'].values
#y_emotion = data['tag'].values
responses = data['response'].values

# Use LabelEncoder to encode emotion labels
label_encoder_emotion = LabelEncoder()
y_emotion_encoded = label_encoder_emotion.fit_transform(y_emotion)

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences)

# Split data into training and validation sets
X_train, X_val, y_emotion_train, y_emotion_val, responses_train, responses_val = train_test_split(
    X_padded, y_emotion_encoded, responses, test_size=0.2, random_state=42
)


# Create emotion classification model
emotion_model = Sequential()
emotion_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X_padded.shape[1]))
emotion_model.add(Bidirectional(LSTM(128)))
#emotion_model.add(Dense(32, activation='relu'))
#emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(len(np.unique(y_emotion_encoded)), activation='softmax'))
emotion_model.compile(optimizer=Adam(learning_rate=0.014), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train emotion classification model
emotion_model.fit(X_train, y_emotion_train, epochs=25, validation_data=(X_val, y_emotion_val))
saved_model_path = "./my_model.h5"
import pickle
pickle.dump(label_encoder_emotion, open('encoder.pkl', 'wb'))
pickle.dump(tokenizer, open('tokenize.pkl', 'wb'))

# YOUR CODE HERE
emotion_model.save(saved_model_path)
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

def preprocess_user_input(text):
    # Remove special characters and multiple spaces
    processed_text = re.sub(r'[^a-zA-Z\s]', '', text)
    processed_text = re.sub(' +', ' ', processed_text).strip()
    return processed_text


# Chatbot function
def chatbot(input_text):
    input_text= preprocess_user_input(input_text)

    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_sequence, maxlen=X_padded.shape[1])

    # Predict emotion label
    predicted_emotion = emotion_model.predict(input_padded)
    predicted_emotion_label = label_encoder_emotion.inverse_transform(np.argmax(predicted_emotion, axis=1))[0]

    # Get responses for the predicted emotion label
    relevant_responses = [response for i, response in enumerate(responses_train) if y_emotion_train[i] == label_encoder_emotion.transform([predicted_emotion_label])[0]]

    # Randomly select a response from relevant_responses
    if len(relevant_responses) > 0:
        selected_response = np.random.choice(relevant_responses)
    else:
        selected_response = "Maaf, saya tidak memiliki respons untuk emosi ini."

    return predicted_emotion_label, selected_response

# Example usage
while True:
    user_input = input("Anda: ")

    if user_input.lower() == 'exit':
        print("Chatbot: Sampai jumpa!")
        break

    predicted_emotion, predicted_response = chatbot(user_input)

    print(f"Chatbot:  {predicted_emotion} -  {predicted_response}")


Epoch 1/25
20/20 [==============================] - 13s 244ms/step - loss: 3.5605 - accuracy: 0.1702 - val_loss: 2.7038 - val_accuracy: 0.3464
Epoch 2/25
20/20 [==============================] - 3s 136ms/step - loss: 1.6637 - accuracy: 0.6072 - val_loss: 1.2085 - val_accuracy: 0.7647
Epoch 3/25
20/20 [==============================] - 2s 125ms/step - loss: 0.6187 - accuracy: 0.8527 - val_loss: 1.0394 - val_accuracy: 0.8301
Epoch 4/25
20/20 [==============================] - 2s 101ms/step - loss: 0.3328 - accuracy: 0.9116 - val_loss: 0.9513 - val_accuracy: 0.8693
Epoch 5/25
20/20 [==============================] - 1s 68ms/step - loss: 0.2007 - accuracy: 0.9362 - val_loss: 1.0072 - val_accuracy: 0.8301
Epoch 6/25
20/20 [==============================] - 1s 67ms/step - loss: 0.1702 - accuracy: 0.9525 - val_loss: 0.9599 - val_accuracy: 0.8431
Epoch 7/25
20/20 [==============================] - 1s 67ms/step - loss: 0.1636 - accuracy: 0.9558 - val_loss: 0.9041 - val_accuracy: 0.8431
Epoch 8/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Anda: haloo
1/1 [==============================] - 1s 782ms/step
Chatbot:  casual -  Ceritakan lebih banyak
Anda: saya lagi sedih ni
1/1 [==============================] - 0s 24ms/step
Chatbot:  sedih -  Mengapa menurut Anda Anda merasa seperti ini?
Anda: pasangan saya tidak mengerti saya
1/1 [==============================] - 0s 42ms/step
Chatbot:  understand -  Saya sedang berusaha yang terbaik untuk membantumu. Jadi, tolong bicaralah padaku.
Anda: Saya merasa tidak dihargai oleh pasangan
1/1 [==============================] - 0s 24ms/step
Chatbot:  Cinta -  Bersyukur atas cinta yang Anda miliki, luangkan waktu bersama dan tunjukkan perhatian.
Anda: Saya merasa cemburu terhadap teman dekat pasangan saya.
1/1 [==============================] - 0s 44ms/step
Chatbot:  Cinta -  Konflik yang berlarut-larut bisa sangat melelahkan. Berbicaralah terbuka dengan pasanganmu, dan pertimbangkan untuk mencari bantuan profesional untuk mendukung pemecahan masalah.
Anda: Aku merasa diabaikan oleh te